In [1]:
import argparse
import os
import matplotlib.pyplot as plt
import torch
import models
from common.fstream import read_tensor, read_matrix, rescale_array

PATH_BASE = "/home/amedvedev/projects/python/DenoisingCNN"
NAME_DATASET = "gcg19"
DEVICE = "cuda:0"

In [2]:
from dataloaders.featureloader2d_v2 import FeatureLoader2d
import torchvision

BATCH_SIZE = 2


transforms = torchvision.transforms.Compose([
        torchvision.transforms.CenterCrop(64),
        torchvision.transforms.Normalize((0.5), (0.5))
    ])

dataset = FeatureLoader2d(
    data_dir=os.path.join(PATH_BASE, "data", "datasets", NAME_DATASET, "train", "calculations"), 
    feature="Uvych2_abs.xls", 
    device=torch.device(DEVICE), 
    batch_size=BATCH_SIZE, 
    shape=(1, 64, 64), 
    read_tensor=read_matrix, 
    transform=transforms, 
    lazy_load=False, 
    pct_load=None, 
    dtype=None,
)
    

In [3]:
from models.diffusion.ddpm_dome272 import ddpm

In [4]:
model = ddpm.UNet(c_in=1, c_out=1, device=DEVICE).to(torch.device(DEVICE))

In [5]:
model.load_state_dict(torch.load("./models/DDPM_Uncondtional/ckpt.pt"))

<All keys matched successfully>

In [6]:
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)

In [8]:
diffusion = ddpm.Diffusion(img_size=64, noise_steps=20, beta_start=1e-9, beta_end=1e-5, device=model.device)

In [10]:
ddpm.train(run_name="DDPM_Uncondtional", 
           diffusion=diffusion,
           dataloader=dataset,
           model=model,
           optimizer=optimizer,
           epochs=1, 
           ch=1,
)


02:18:24 - INFO: Starting epoch 0:
100%|██████████| 447/447 [11:36<00:00,  1.56s/it, MSE=0.109] 
02:30:01 - INFO: Sampling 2 new images....
19it [00:11,  1.62it/s]


In [8]:
noise_steps_old = diffusion.noise_steps
diffusion.noise_steps=200
x = diffusion.sample(model, 1, 1)

plt.figure(figsize=(32, 32))
plt.imshow(torch.cat([
    torch.cat([i for i in x.cpu()], dim=-1),
], dim=-2).permute(1, 2, 0).cpu(), cmap="jet")
diffusion.noise_steps=noise_steps_old
plt.show()

01:44:47 - INFO: Sampling 1 new images....
0it [00:00, ?it/s]../aten/src/ATen/native/cuda/IndexKernel.cu:92: operator(): block: [0,0,0], thread: [0,0,0] Assertion `-sizes[i] <= index && index < sizes[i] && "index out of bounds"` failed.
0it [00:00, ?it/s]


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [1]:
import gc, torch
gc.collect()
torch.cuda.empty_cache()